# Topic 1. Agent and Environment

# 1-1. Import python package

In [1]:
import math
import sys

# 1-2. Problem abstract class definition
For robotic field, we often call an **agent** is anything that can be viewed as perceiving its **environment** through **sensors** and acting upon that environment through **actuators**.

<img src="images/goal_based_reflex_agent.png" width="480px">

We start by defining the abstract class for a `Problem`; specific problem domains will subclass this, and then you can create individual problems with `specific initial states` and `goals`. you can also define a `Node` in a search tree to slove the problem. We will talk it later.

In [2]:
class Problem(object):
    '''The abstract class for a formal problem. You should subclass this,
    overriding `actions` and `results`, and other methods if necessary.
    Note: a problem can specify a default heuristic if desired. By default, 
    the heuristic is 0 for all states, and the step cost is 1 for all actions.'''

    def __init__(self, initial=None, goal=None, **other_keywords):
        '''Specify the initial and goal states.
        Subclasses can use other keywords if they want.'''
        self.__dict__.update(initial=initial, goal=goal, **other_keywords) 

    def actions(self, state):
        '''Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once.'''
        raise NotImplementedError
        
    def result(self, state, action):
        '''Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state).'''
        raise NotImplementedError
        
    def is_goal(self, state):
        '''True if the state is a goal.'''
        return state == self.goal
    
    def step_cost(self, state1, action, state2):
        '''Return the cost of a solution path that arrives at state2 from
        state1 via action. The default method 
        costs 1 for every step in the path.'''
        return 1
    
    def h(self, node):
        '''Return the cost of a solution path that arrives at goal state from
        current state. The default value is 0'''
        return 0
    

## Example1-1 problem definition
Take example, a Vacuum Robot in a world with two locations, and dirt as shown below. Each state is a tuple of (location, dirt_in_L, dirt_in_R).
<img src="images/vacuum_problem.jpg">

In [3]:
'''
First, we define the state with tuple data type, 
just like ('L', '*', ' '). It means the vacuum
robot is on Left side, the place is dirty on 
left side and clean on right side.
'''

dirt  = '*'
clean = ' '

class TwoLocationVacuumProblem(Problem):    
    def actions(self, state): return ('L', 'R', 'S')

    def result(self, state, action):
        '''The state that results from executing this action in this state. '''      
        (loc, dirtL, dirtR) = state
        if   action == 'L':                   return ('L', dirtL, dirtR)
        elif action == 'R':                   return ('R', dirtL, dirtR)
        elif action == 'S' and loc == 'L': return (loc, clean, dirtR)
        elif action == 'S' and loc == 'R': return (loc, dirtL, clean) 
        else: raise ValueError('unknown action: ' + action)

In [4]:
initial_state=('R', '*', '*')
p1 = TwoLocationVacuumProblem(initial=initial_state)
print('Initial state of the problem:  {}' .format(p1.initial))

# Show what's the result from excuting the action 'Suck' in the initial state.
result1 = p1.result(state=p1.initial, action=('S'))
print('Result by excuting the action: {}' .format(result1))

Initial state of the problem:  ('R', '*', '*')
Result by excuting the action: ('R', '*', ' ')


==================================================================================================================

# 1-3. Node definition
We also ddefine a `Node` in a search tree, and some functions on nodes: `expand` to generate successors, and `path_actions`, `path_states` and `path` to recover aspects of the path from the node.

In [5]:
class Node:
    '''A Node in a search tree.'''
    def __init__(self, state, parent=None, action=None, path_cost=0):
        # __dict__ store this object's all attributes
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
    
    '''All Reserve words are not introduced here. If you are interest in them, please Google them'''
    # __repr__ is a built-in function used to compute the '''official''' string reputation of an object.
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.state < other.state
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deeepening search was cut off.

def expand(problem, node):
    '''Expand a node, generating the children nodes.'''
    s = node.state
    for action in problem.actions(s): 
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.step_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    '''The sequence of actions to get to this node.'''
    if node.parent is None:
        return []
    else: 
        return path_actions(node.parent) + [node.action]


def path_states(node):
    '''The sequence of states to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path_states(node.parent)) + [node.state]


def path(node):
    '''Alternating states and actions to get to this node.'''
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path(node.parent) + [node.action] ) + [node.state]

## Example1-2 DFS 
We quicklly define a DFS algorithm with step limitation. 

In [6]:
def depth_limited_search(problem, limit=5):
    '''Search deepest nodes in the search tree first.'''
    frontier = list([Node(problem.initial)])
    solution = failure
    while frontier:
        node = frontier.pop()
        if len(node) > limit:
            solution = cutoff
        else:
            for child in expand(problem, node):
                if problem.is_goal(child.state):
                    return child
                frontier.append(child)
    return solution

In [7]:
initial_state=('R', '*', '*')
goal_state=('L', ' ', ' ')
p1 = TwoLocationVacuumProblem(initial=initial_state, goal=goal_state)

# Apply the DFS on the vacuum problem
result_graph = depth_limited_search(problem=p1, limit=2)

# Take a look the state sequence of the result
path_states(result_graph)

[('R', '*', '*'), ('R', '*', ' '), ('L', '*', ' '), ('L', ' ', ' ')]

In [8]:
# Take a look the action sequence of the result
path_actions(result_graph)

['S', 'L', 'S']